<a href="https://colab.research.google.com/github/yiwenwangANU/pytorch_review/blob/main/02_Multiclass_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Preparing

In [1]:
import kagglehub
# https://www.kaggle.com/datasets/mujtabamatin/air-quality-and-pollution-assessment/
# Download latest version
path = kagglehub.dataset_download("mujtabamatin/air-quality-and-pollution-assessment")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/mujtabamatin/air-quality-and-pollution-assessment/versions/2


In [2]:
import os
print(os.listdir(path))

['updated_pollution_dataset.csv']


In [3]:
csv_path = os.path.join(path, "updated_pollution_dataset.csv")

In [4]:
import pandas as pd
# Read file as data frame
df = pd.read_csv(csv_path)

In [5]:
# Data visualization
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Temperature                    5000 non-null   float64
 1   Humidity                       5000 non-null   float64
 2   PM2.5                          5000 non-null   float64
 3   PM10                           5000 non-null   float64
 4   NO2                            5000 non-null   float64
 5   SO2                            5000 non-null   float64
 6   CO                             5000 non-null   float64
 7   Proximity_to_Industrial_Areas  5000 non-null   float64
 8   Population_Density             5000 non-null   int64  
 9   Air Quality                    5000 non-null   object 
dtypes: float64(8), int64(1), object(1)
memory usage: 390.8+ KB


In [6]:
df.head(10)

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,Air Quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,Moderate
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,Moderate
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,Moderate
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,Good
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,Good
5,39.4,96.6,14.6,35.5,42.9,17.9,1.82,3.1,674,Hazardous
6,41.7,82.5,1.7,15.8,31.1,12.7,1.80,4.6,735,Poor
7,31.0,59.6,5.0,16.8,24.2,13.6,1.38,6.3,443,Moderate
8,29.4,93.8,10.3,22.7,45.1,11.8,2.03,5.4,486,Poor
9,33.2,80.5,11.1,24.4,32.0,15.3,1.69,4.9,535,Poor


In [7]:
# Encode air quality column
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Air Quality'] = encoder.fit_transform(df['Air Quality'])

In [8]:
# Check the data
df.head(10)

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,Air Quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,2
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,2
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,2
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,0
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,0
5,39.4,96.6,14.6,35.5,42.9,17.9,1.82,3.1,674,1
6,41.7,82.5,1.7,15.8,31.1,12.7,1.80,4.6,735,3
7,31.0,59.6,5.0,16.8,24.2,13.6,1.38,6.3,443,2
8,29.4,93.8,10.3,22.7,45.1,11.8,2.03,5.4,486,3
9,33.2,80.5,11.1,24.4,32.0,15.3,1.69,4.9,535,3


In [9]:
# Create X and y from data frame
# Using 'Air Quality' as target value and all others as the features
X = df.drop('Air Quality', axis=1).values
y = df['Air Quality'].values

In [10]:
# Check the data
X[:5]

array([[ 29.8 ,  59.1 ,   5.2 ,  17.9 ,  18.9 ,   9.2 ,   1.72,   6.3 ,
        319.  ],
       [ 28.3 ,  75.6 ,   2.3 ,  12.2 ,  30.8 ,   9.7 ,   1.64,   6.  ,
        611.  ],
       [ 23.1 ,  74.7 ,  26.7 ,  33.8 ,  24.4 ,  12.6 ,   1.63,   5.2 ,
        619.  ],
       [ 27.1 ,  39.1 ,   6.1 ,   6.3 ,  13.5 ,   5.3 ,   1.15,  11.1 ,
        551.  ],
       [ 26.5 ,  70.7 ,   6.9 ,  16.  ,  21.9 ,   5.6 ,   1.01,  12.7 ,
        303.  ]])

In [11]:
from sklearn.preprocessing import StandardScaler
# Standardization
scaler = StandardScaler()
X = scaler.fit_transform(X)
X[:5]

array([[-0.03408041, -0.69071534, -0.60858932, -0.45045531, -0.84458126,
        -0.12072072,  0.40230264, -0.58865846, -1.16816284],
       [-0.25729505,  0.34950721, -0.72670554, -0.65889176,  0.49332918,
        -0.04664257,  0.25577499, -0.67174754,  0.74359753],
       [-1.03110581,  0.2927678 ,  0.26709986,  0.13097269, -0.22621929,
         0.38301072,  0.23745904, -0.89331843,  0.79597453],
       [-0.43586677, -1.95159116, -0.57193257, -0.87464177, -1.45170029,
        -0.69853032, -0.64170683,  0.74076687,  0.35077006],
       [-0.52515262,  0.04059264, -0.53934878, -0.51993413, -0.50729292,
        -0.65408342, -0.8981302 ,  1.18390865, -1.27291684]])

In [12]:
# Check Data Imbalance
import torch
class_counts = df['Air Quality'].value_counts().sort_index()
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
class_counts

,count
Air Quality,
0,2000
1,500
2,1500
3,1000


In [13]:
# Turn numpy data into tensors

X, y = torch.from_numpy(X).type(torch.float), torch.from_numpy(y).type(torch.LongTensor)

In [14]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [15]:
# Check the data
X_train[:5], X_train.dtype, len(X_train)

(tensor([[ 1.3052,  1.4149, -0.4823, -0.0446,  0.7182, -0.2541,  1.2632, -1.2534,
           1.3132],
         [-0.5549, -1.0059, -0.7837, -0.7284, -0.1475, -0.8171, -1.0080,  0.4638,
          -1.1878],
         [ 1.0671,  2.6002, -0.8122, -0.7832,  0.6170,  1.2571,  1.1349, -0.2009,
           1.6733],
         [-1.8644, -0.8105, -0.4660, -0.6077, -0.0913, -1.0245, -0.8615,  0.8792,
           0.0496],
         [-0.4954, -0.9618, -0.5842, -0.6918, -1.1931, -0.8171, -0.4585,  0.9346,
           0.4424]]),
 torch.float32,
 4000)

In [16]:
y_train[:5], y_train.dtype, len(y_train)

(tensor([1, 0, 3, 0, 0]), torch.int64, 4000)

##Set up GPU

In [17]:
# need to switch to GPU in colab first if wants GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [18]:
# put data set onto device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

##Build the model

In [19]:
from torch import nn
class AirPollutionModel(nn.Module):
  def __init__(self, in_features, out_features, hidden_units=64):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Linear(in_features=in_features, out_features=hidden_units),
        nn.ReLU(),
        nn.Dropout(0.3), # add drop out layer to avoid overfitting
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(in_features=hidden_units, out_features=out_features)
    )

  def forward(self, x):
    return self.layer_stack(x)

model = AirPollutionModel(in_features=X_train.shape[1], out_features=len(y_train.unique())).to(device)

In [20]:
loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = torch.optim.Adam(params=model.parameters(), lr=.01)

##Train the model

In [21]:
epoches = 3000
for epoch in range(epoches):
  model.train()
  y_logits = model(X_train)
  loss = loss_fn(y_logits, y_train)
  y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)
  acc = (y_pred == y_train).sum().item() / len(y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  model.eval()
  with torch.inference_mode():
    if(epoch % 300 == 0):
      test_logits = model(X_test)
      test_loss = loss_fn(test_logits, y_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      test_acc = (test_pred == y_test).sum().item() / len(y_test)
      print(f"Epoch: {epoch} | Train Loss: {loss} | Test Loss: {test_loss} | Train Accuracy: {acc} | Test Accuracy: {test_acc}")


Epoch: 0 | Train Loss: 1.3770256042480469 | Test Loss: 1.2261015176773071 | Train Accuracy: 0.35325 | Test Accuracy: 0.596
Epoch: 300 | Train Loss: 0.13670845329761505 | Test Loss: 0.15634940564632416 | Train Accuracy: 0.95925 | Test Accuracy: 0.953
Epoch: 600 | Train Loss: 0.11388349533081055 | Test Loss: 0.20099012553691864 | Train Accuracy: 0.966 | Test Accuracy: 0.944
Epoch: 900 | Train Loss: 0.10180647671222687 | Test Loss: 0.22222010791301727 | Train Accuracy: 0.97075 | Test Accuracy: 0.95
Epoch: 1200 | Train Loss: 0.0906560868024826 | Test Loss: 0.2149810492992401 | Train Accuracy: 0.972 | Test Accuracy: 0.948
Epoch: 1500 | Train Loss: 0.09736648947000504 | Test Loss: 0.26753464341163635 | Train Accuracy: 0.971 | Test Accuracy: 0.951
Epoch: 1800 | Train Loss: 0.08733829855918884 | Test Loss: 0.30324503779411316 | Train Accuracy: 0.9735 | Test Accuracy: 0.952
Epoch: 2100 | Train Loss: 0.08254072070121765 | Test Loss: 0.30916526913642883 | Train Accuracy: 0.976 | Test Accuracy: 0.